In [5]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from qtorch_plus.quant import posit_quantize
import requests
import zipfile

In [7]:
# Set up a local path for the dataset
dataset_dir = "/localsdd/yg9bq/LLM_quantization/imagenet_val"  # Target directory for the validation dataset

# Ensure the dataset directory exists and contains the required files
if not os.path.exists(dataset_dir):
    print(f"Dataset directory '{dataset_dir}' not found. Downloading ImageNet Mini...")
    
    # Download ImageNet Mini
    url = "https://github.com/Divya-John/imagenet-mini/archive/refs/heads/main.zip"
    zip_path = "./imagenet-mini.zip"
    response = requests.get(url, stream=True)
    with open(zip_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    print("Download completed.")
    
    # Extract the dataset
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall("./")
    
    # Move the validation dataset to the target directory
    os.rename("./imagenet-mini-main/val", dataset_dir)
    os.rmdir("./imagenet-mini-main")  # Clean up the unzipped folder
    os.remove(zip_path)  # Clean up the zip file
    print(f"Dataset extracted to '{dataset_dir}'.")

# Define the dataset and preprocessing
print("Loading ImageNet dataset...")
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

imagenet_dataset = torchvision.datasets.ImageFolder(dataset_dir, transform=transform)
val_loader = DataLoader(imagenet_dataset, batch_size=32, shuffle=False, num_workers=4)
print("ImageNet validation dataset loaded.")

Dataset directory '/localsdd/yg9bq/LLM_quantization/imagenet_val' not found. Downloading ImageNet Mini...
Download completed.


BadZipFile: File is not a zip file

Loading ResNet-100 model...


/home/yg9bq/miniconda3/envs/posit/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/yg9bq/miniconda3/envs/posit/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /home/yg9bq/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:03<00:00, 57.4MB/s] 


Model loaded successfully.


In [3]:
# Define quantization functions
def linear_weight(input):
    return posit_quantize(input, nsize=8, es=1, scale=1)

def linear_activation(input):
    return posit_quantize(input, nsize=8, es=1, scale=1)

def forward_pre_hook_linear(m, input):
    return (linear_activation(input[0]),)

# Apply quantization to all layers
layer_count = 0
conv_count = 0
op_count = 0

for name, module in resnet100.named_modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        layer_count += 1

        # Quantize weights
        print(f"Quantizing layer {name}")
        module.weight.data = linear_weight(module.weight.data)

        # Register forward hook for activations
        module.register_forward_pre_hook(forward_pre_hook_linear)

        # Count operations
        if isinstance(module, nn.Conv2d):
            conv_count += 1
            op_count += module.weight.shape[0] * module.weight.shape[1] * module.weight.shape[2] * module.weight.shape[3]
        elif isinstance(module, nn.Linear):
            op_count += module.in_features * module.out_features

print("MAC operation count:", op_count)
print("Total layers quantized:", layer_count)

Quantizing layer conv1
Quantizing layer layer1.0.conv1
Quantizing layer layer1.0.conv2
Quantizing layer layer1.0.conv3
Quantizing layer layer1.0.downsample.0
Quantizing layer layer1.1.conv1
Quantizing layer layer1.1.conv2
Quantizing layer layer1.1.conv3
Quantizing layer layer1.2.conv1
Quantizing layer layer1.2.conv2
Quantizing layer layer1.2.conv3
Quantizing layer layer2.0.conv1
Quantizing layer layer2.0.conv2
Quantizing layer layer2.0.conv3
Quantizing layer layer2.0.downsample.0
Quantizing layer layer2.1.conv1
Quantizing layer layer2.1.conv2
Quantizing layer layer2.1.conv3
Quantizing layer layer2.2.conv1
Quantizing layer layer2.2.conv2
Quantizing layer layer2.2.conv3
Quantizing layer layer2.3.conv1
Quantizing layer layer2.3.conv2
Quantizing layer layer2.3.conv3
Quantizing layer layer3.0.conv1
Quantizing layer layer3.0.conv2
Quantizing layer layer3.0.conv3
Quantizing layer layer3.0.downsample.0
Quantizing layer layer3.1.conv1
Quantizing layer layer3.1.conv2
Quantizing layer layer3.1.co

In [4]:
# Optionally test the quantized model on some data
from torchvision.transforms import Compose, ToTensor, Normalize
from PIL import Image

# Example input image preprocessing
def preprocess_image(image_path):
    transform = Compose([
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).to("cuda")

# Load a sample image and pass it through the quantized model
example_image_path = "path/to/sample_image.jpg"  # Replace with an actual image path
input_image = preprocess_image(example_image_path)

# Perform inference
resnet100.eval()
with torch.no_grad():
    output = resnet100(input_image)
    print("Model inference completed.")
    print("Output shape:", output.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/sample_image.jpg'